## Data collection

Defining the function for data collection from the links of HTML pages, and storing in TSV files

### Collector_Utils

In [ ]:
import os.path

def PRINT(save_path, counter, webContent):
    # We are going to create an .html file into the folder with the page
    completeName = os.path.join(save_path, 'article_' + str(counter) + '.html')
    with open(completeName, 'w') as out_file:
        out_file.write(str(webContent))
    return

#### Main Collector file for Crawling the data and fetching fuction from Collector_Utils for Storing.

### Collector

In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import urllib.request, urllib.error, urllib.parse
from collector_utils import PRINT
import numpy as np
import pickle

#### Assigning Variable folder where we want save all the html files, for Counter of File extraction status and emplty list

In [ ]:
# where we want save all the html files
save_path = '/home/lex/Desktop/Data_science/Algorithmic_Methods_of_Data_Mining/ADM_hw3/'


# counter for the name of the file
counter = 0

# a list of all the empty pages
empty_file = []

# a dictionary with as key the Id of the document and walue his url
Doc_Id_url = {}

#### Generating a loops with in loop for the following task

1.To access all HTML file 1 after another from 1 to 3

2.Parsing the links (wiki_links) from these, and converting them into a SOUP

3.Reading the WEB content and producing HTML file in HTML Folder

4.Extracting the data

5.Fectching PRINT Function from Collector_Utils

6.Saving the HTML data in an HTMl folder

7.Counter Ranging from 1 to 5 Secs

8.Creating a dictionary or URL

In [ ]:
# A loop for each list of link
for i in range(1,4):
    
    # We are reading the HTML with all the links for the pages of wikipedia
    list_of_pages = 'https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies' + str(i) + '.html'               
    request = requests.get(list_of_pages )
    
    
    # we parse this file 
    soup = BeautifulSoup(request.text, 'html.parser')
    
    # we create a list with all the links
    wiki_links = soup.select('a')
    
    N = len(wiki_links)
    for j in range(0,N):
        # save the url and the link
        Doc_Id_url[counter] = wiki_links[j].get('href')
        
        try:
            print(i,j,counter)
            
            # read the j-th page of wikipedia in the i-th HTML page of all links
            response = urllib.request.urlopen(wiki_links[j].get('href'))
            webContent = response.read().decode(response.headers.get_content_charset())
            
            # this function save this page in an .html file inside the folder at the address save_path 
            PRINT(save_path +'HTML', counter, webContent)
            soup_i = BeautifulSoup(webContent, 'html5lib')
            
        # lets see the exceptions
        except urllib.error.HTTPError as e:
            # too many request so sleep for 20min and after save the last page
            if e.code == 429:
                time.sleep(20*60)
                response = urllib.request.urlopen(wiki_links[j].get('href'))
                webContent = response.read().decode(response.headers.get_content_charset())
                PRINT(save_path +'HTML', counter, webContent) 
            # page not found so save an html with the write 'NA'
            if e.code == 404:
                PRINT(save_path + 'HTML', counter, 'NA')
                empty_file.append(counter)
              
        
        counter += 1
        # andom time sleep between 1 and 5 sec
        time.sleep(np.random.randint(1,6))
        

file = '/home/lex/Desktop/Data_science/Algorithmic_Methods_of_Data_Mining/ADM_hw3/'

# save the dictionary as pkl
with open(file+'Doc_Id_url.pkl', 'wb') as fp:
    pickle.dump(Doc_Id_url, fp, protocol = pickle.HIGHEST_PROTOCOL)


### Parsing
For parsing we have to extract the INTRO, PLOT and INFOBOX for all related information

so for this purpose we made a PARSING_UTILS a function Library for parsing the DATA

### PARSING_UTILS

In [ ]:
def PLOT(tag_i, plot):
    # from the <h2> until there is a new <h2> we take all the plots
    try:
        while True:
            if tag_i.name == 'h2':
                return plot
            if tag_i.name == 'p':
                plot.append(tag_i.text[:-2].replace('\n', ''))
            tag_i = tag_i.find_next_sibling()
    except: 
        # it's for pages that have nothing after the polt
        pass
    return plot

def INTRO(all_p_i,plot,intro):
    # if plot in empty we try to save the first paragraph, otherwise we save 'NA'
    if plot=='NA':
        try:
            return all_p_i[0][-2]
        except:
            return 'NA'
    #select all the paragraph writen after the plot
    else:
        for par in all_p_i:
            if par.text[:-2].replace('\n', '') == plot[0]:
                break
            else:
                intro.append(par.text.replace('\n', ''))
    return intro


def DICT_INFOBOX(soup_i,result):
    # we read the infobox and make a dictionary
    table = soup_i.find('table', class_='infobox vevent')
    for tr in table.find_all('tr'):
        if tr.find('th'):
            result[tr.find('th').text] = tr.get_text(strip=True, separator=" ")[len(tr.find('th').text):]#[m.text for m in tr.find_all('td')]#[len(tr.find('th')):]
        else:
            pass
    return result

def EMPTY_INFOBOX():
    # if there is not the infobox we save a dictionary with only 'NA' for each key
    a = {'Directed by': 'NA',
             'Produced by': 'NA',
             'Written by': 'NA',
             'Starring': 'NA',
             'Music by': 'NA',
             'Release date': 'NA',
             'Running time': 'NA',
             'Country': 'NA',
             'Language': 'NA',
             'Budget': 'NA'}
    return a



1.Reading from HTML Folder HTML Files

2.Extracting the Required data via fetching the function from PARSING_UTILS

3.making TSV Output in a TSV folder

### PARSING

In [ ]:
from bs4 import BeautifulSoup
import csv
from parse_utils import PLOT, INTRO, DICT_INFOBOX, EMPTY_INFOBOX
import os.path



# read the html files from the HTML folder and save the tvs files inside the folder TSV
input_path = '/home/lex/Desktop/Data_science/Algorithmic_Methods_of_Data_Mining/ADM_hw3/HTML'
output_path = '/home/lex/Desktop/Data_science/Algorithmic_Methods_of_Data_Mining/ADM_hw3/TSV'


# how wiki call the plot section
possible_plots = ['#Plot','#Plot_summary', '#Premise']
     
# how save informatrions into the tsv files                  
infos = ['title', 'intro', 'plot' ,'film_name', 'director', 'producer', 'writer', 'starring', 'music', 'release date', 'runtime', 'country', 'language', 'budget']

# what wiki write inside the infobox
Name = ['Directed by', 'Produced by', 'Written by', 'Starring', 'Music by', 'Release date', 'Running time', 'Country', 'Language', 'Budget']

# the list of all the empty html page
b = []


# read file from i up to N
N = 30000
i = 0

while i < N:
    print(i)
    # select the html file
    completeName_input = os.path.join(input_path, 'article_' + str(i) + '.html')
    try:
        
        # open that file
        with open(completeName_input, 'r') as out_file:
            page = out_file.read() 
    
        soup_i = BeautifulSoup(page, 'html5lib')
        
        # read all the paragraph
        all_p_i = soup_i.find_all('p')
       
        
        
        # for each possible name of the plot we do the follow loop
        no_plot = 0
        for k in possible_plots:
            try:
                # we are going to select the <h2> associated to the k-th name of the plot
                # if we find it we create the plot with the function PLOT inside collectop_utils.py
                tag_i = soup_i.select_one(k).find_parent('h2').find_next_sibling()
                plot = PLOT(tag_i, [])
                no_plot = 1
                break                         
            except:
                pass
        # if there is no plot we write 'NA'
        if no_plot == 0:
            plot = 'NA'
        
        # reading the intro: everything before the plot
        intro = INTRO(all_p_i,plot,[])    
        
        # try to read the infobox
        try: 
            # if we find it whe read it and save it as a dictionary into the result
            table = soup_i.find('table', class_='infobox vevent')
            result = DICT_INFOBOX(soup_i,{})
        except:
            # otherwise we save a dictionary with only 'NA' for each key
            result = EMPTY_INFOBOX()
        
        
        # we are going to create an array with only 0
        InfoFilm= [0]*14
        
        # saving the infobox
        for k in range(10):
            InfoFilm[k+4] = result.get(Name[k], 'NA')
        # saving the title
        InfoFilm[0] = soup_i.title.text[:-12]
        
        # saving the film_name
        try:
            InfoFilm[3] = table.find_all('tr')[0].text
        except:
            InfoFilm[3] = 'NA'
        
        # saving plot and intro
        InfoFilm[2] = ' '.join(plot)
        InfoFilm[1] = ' '.join(intro)
        
        # save the tsv file
        completeName_output = os.path.join(output_path, 'article_' + str(i) + '.tsv')
        
        with open(completeName_output, 'wt') as out_file:
            tsv_writer = csv.writer(out_file, delimiter='\t')
            tsv_writer.writerow(InfoFilm)
        
    except:
        # if the html in empty, this mean that in collector there was the exception 404 so we save only 'NA' for each informations
        InfoFilm = ['NA']*14
        completeName_output = os.path.join(output_path, 'article_' + str(i) + '.tsv')
        with open(completeName_output, 'wt') as out_file:
            tsv_writer = csv.writer(out_file, delimiter='\t')
            tsv_writer.writerow(InfoFilm)
        
        b.append(i)

    
    i += 1

## SEARCH ENGINE

#### 1.Cleaning of Data
Tokenizing | Removal of Stop Words | Stemming
Using nltk library

#### 2.Indexing of Data
Indexing and INVERTED indexing

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from collections import Counter
import numpy as np

# generate the stop-words
stopWords = set(stopwords.words('english'))

# this function work for a generic string, such as a document or the query
def FILTER(string):
    # we are going to filter the string
    string = string.lower()
    string = string.replace("'", "")
    # tokenize the string
    tokenizer = RegexpTokenizer("[\w']+")
    Vector = tokenizer.tokenize(string)
    # removing the stop-words
    wordsFiltered = []
    for w in Vector:
        if w not in stopWords:
            wordsFiltered.append(w)
    #stemmer
    porter = PorterStemmer()
    ListOfWords = [porter.stem(word) for word in wordsFiltered]
    return ListOfWords

#### 2.Creating Direct INDEX

In [ ]:
# we use this function to create the direct index
# is a dictionry with as key the word and as value another dictionary with key the number of the document and as value the frequency of the word in this document
def DOC_DOCLEN_FREQ_DIC(Document, direct_index, i):
    Document = Counter(Document)
    for word, frequence in Document.items():
        if word not in direct_index:
            direct_index[word] = {i: frequence}
        else:
            direct_index[word][i] = frequence
    return 

#### 3.Creating INVERTED INDEX

In [ ]:
# we use this function to create the inverted index 
def INVERTED_INDEX_DIC(direct_index, Inverted_index, Length_Doc, Number_of_documents):
    for key, Doc_Freq in direct_index.items():
        Inverted_index[key] = {}
        for doc, freq in Doc_Freq.items():
            if key not in Inverted_index:
                Inverted_index[key] = {doc: (freq/ Length_Doc[doc])*np.log10(Number_of_documents/len(Doc_Freq))}
            else:
                Inverted_index[key][doc] = (freq/ Length_Doc[doc])*np.log10(Number_of_documents/len(Doc_Freq))
    
    return 

#### Combining all these three functions in INDEX_UTILS

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from collections import Counter
import numpy as np

# generate the stop-words
stopWords = set(stopwords.words('english'))

# this function work for a generic string, such as a document or the query
def FILTER(string):
    # we are going to filter the string
    string = string.lower()
    string = string.replace("'", "")
    # tokenize the string
    tokenizer = RegexpTokenizer("[\w']+")
    Vector = tokenizer.tokenize(string)
    # removing the stop-words
    wordsFiltered = []
    for w in Vector:
        if w not in stopWords:
            wordsFiltered.append(w)
    #stemmer
    porter = PorterStemmer()
    ListOfWords = [porter.stem(word) for word in wordsFiltered]
    return ListOfWords

# we use this function to create the direct index
# is a dictionry with as key the word and as value another dictionary with key the number of the document and as value the frequency of the word in this document
def DOC_DOCLEN_FREQ_DIC(Document, direct_index, i):
    Document = Counter(Document)
    for word, frequence in Document.items():
        if word not in direct_index:
            direct_index[word] = {i: frequence}
        else:
            direct_index[word][i] = frequence
    return 

# we use this function to create the inverted index 
def INVERTED_INDEX_DIC(direct_index, Inverted_index, Length_Doc, Number_of_documents):
    for key, Doc_Freq in direct_index.items():
        Inverted_index[key] = {}
        for doc, freq in Doc_Freq.items():
            if key not in Inverted_index:
                Inverted_index[key] = {doc: (freq/ Length_Doc[doc])*np.log10(Number_of_documents/len(Doc_Freq))}
            else:
                Inverted_index[key][doc] = (freq/ Length_Doc[doc])*np.log10(Number_of_documents/len(Doc_Freq))
    
    return 



### Index
Reading Data from TSV file

Creating Vocablury

Saving the Dictionary and Inverted INDEX as .pkl

In [ ]:
import os.path
from index_utils import FILTER, DOC_DOCLEN_FREQ_DIC, INVERTED_INDEX_DIC
import pickle



# we read the tsv files from the folder TSV 
input_path = '/home/lex/Desktop/Data_science/Algorithmic_Methods_of_Data_Mining/ADM_hw3/TSV'


## there are dictionary of dictionary because is easy to read
'''
{ word_id : {doc_id : tfidf,
             doc_id : tfidf,
             doc_id : tfidf }, 
  word _id: {doc_id : tfidf,
             doc_id : tfidf,
             doc_id : tfidf,
             doc_id : tfidf}
  }
'''

# is a dictionry with as key the word and as value another dictionary with key the number of the document and as value the frequency of the word in this document
direct_index = {}

#the inverted index with as key the words
Inverted_index_words = {}

#the inverted index with as key the words_number read from the Vocabulary
Inverted_index = {}

# as key the word as value the number
Vocabulary = {}

# as key the number of the document as value his length after cleaning
Length_Doc = {}

# 30000 - number of empty docs
Number_of_documents = 29982

N = 30000
i = 0
while i < N:
    print(i)
    # read the i-th tsv
    completeName_input = os.path.join(input_path, 'article_' + str(i) + '.tsv')
    with open(completeName_input, 'r') as input_file:
        intro_plot = input_file.readline()
    # select only the intro and the plot
    intro_plot = intro_plot.split('\t')[1:3]
    # make a string
    intro_plot = str(intro_plot[0]) + ' ' + str(intro_plot[1])
    # filter this string
    Document = FILTER(intro_plot)
    # save the length of the document
    Length_Doc[i] = len(Document) 
    
    # make the direct index
    DOC_DOCLEN_FREQ_DIC(Document, direct_index, i)
    
    
    
    i += 1

# save the inverted index with the words as key
INVERTED_INDEX_DIC(direct_index, Inverted_index_words, Length_Doc, Number_of_documents)

# create the Vocabulary and the real inverted index
counter = 0
for i,j in Inverted_index_words.items():
    Vocabulary[i] = counter
    Inverted_index[counter] = j
    counter += 1


c = []

for i in  Inverted_index_words:
    c.append(i)
    
file = '/home/lex/Desktop/Data_science/Algorithmic_Methods_of_Data_Mining/ADM_hw3/'

# save the 3 dictionaries as pkl
with open(file+'Inverted_index.pkl', 'wb') as fp:
    pickle.dump(Inverted_index, fp, protocol = pickle.HIGHEST_PROTOCOL)
    
with open(file+'Vocabulary.pkl', 'wb') as fp:
    pickle.dump(Vocabulary, fp, protocol = pickle.HIGHEST_PROTOCOL)

with open(file+'Length_doc.pkl', 'wb') as fp:
    pickle.dump(Length_Doc, fp, protocol = pickle.HIGHEST_PROTOCOL)

# SEARCH Engine 



In [ ]:
import numpy as np
from index_utils import FILTER
import pickle
from functools import reduce
import pandas as pd
import os.path

# folder with the tsv files
input_path = '/home/lex/Desktop/Data_science/Algorithmic_Methods_of_Data_Mining/ADM_hw3/TSV'

# reading the dictionaries
file = '/home/lex/Desktop/Data_science/Algorithmic_Methods_of_Data_Mining/ADM_hw3/'
with open(file + 'Inverted_index.pkl', 'rb') as fp:
    Inverted_index = pickle.load(fp)

with open(file + 'Vocabulary.pkl', 'rb') as fp:
    Vocabulary = pickle.load(fp)

with open(file + 'Doc_Id_url.pkl', 'rb') as fp:
    Doc_Id_url = pickle.load(fp)

Length = {doc_id :[] for doc_id in range(30000) }
for word_id,doc_id_tfidf in Inverted_index.items():
    for doc_id, tfidf in doc_id_tfidf.items():
        Length[doc_id].append(tfidf)

### Definitions

#### Search Engine 1

In [ ]:
def conjuntive_query(Query, Inverted_index, Vocabulary, Docs):
    for i in Query:
        # select all the docs with a specific word and appen this array to another array
        try:
            Docs.append(np.fromiter(Inverted_index[Vocabulary[i]].keys(), dtype=int))
        except: 
            pass
    # make the and
    if len(Docs) > 0:
        Films = reduce(np.intersect1d, Docs)
    else:
        Films = []
    return Films



def output_conjuntive_query(Films, input_path, Doc_Id_url):
    Title = []
    Intro = []
    Url = []
    # for each film selected we are going to save title, intro and url
    for i in Films:
        completeName_input = os.path.join(input_path, 'article_' + str(i) + '.tsv')
        with open(completeName_input, 'r') as input_file:
            document = input_file.readline()
        document= document.split('\t')
        Title.append(document[0])
        Intro.append(document[1])
        Url.append(Doc_Id_url[i])
        
    #create a dictionary
    Data = {'title' : Title,
            'intro' : Intro,
            'url' : Url
            }
   
    # make the dataframe 
    df = pd.DataFrame(data=Data)
    
    return df



# input for the user
while True:
    try:
        Choice = int(input('which search: 1, 2 or 3?'))
        if Choice == 3:
            # maybe somthing as 'do you want specify the director?'
            pass
        else:
            Query = input('amazing: the query is?')
        break
    except:
        print('try again')



#### SEARCH Engine  2

#### SEARCH Engine  3

In [ ]:
def search_composer_director(Films, input_path):
    # reading into the infobox the composer and the director
    Composers = []
    Directors = []
    if len(Films) != 0:
        for i in Films:
            completeName_input = os.path.join(input_path, 'article_' + str(i) + '.tsv')
            with open(completeName_input, 'r') as input_file:
                document = input_file.readline()
            document= document.split('\t')
            # filtering the words and adding to the array
            Composers.append(FILTER(document[8]))
            Directors.append(FILTER(document[4]))
    else:
        # if there are no movies with all the words in the query
        print('\n\n\nnothing for you')
    return Composers, Directors

def FREQ_value(Inverted_index, Vocabulary, Films):
    # we are interested in the frequency ie tf
    freq = []
    if len(Films) != 0:
        for i in Query:
            freq_word = []
            for j in Films:
                # select all the docs with a specific word and appen this array to another array
                try:
                    freq_word.append(Inverted_index[Vocabulary[i]][j]/np.log10(29982/len(Inverted_index[Vocabulary[i]])))
                except:
                    freq_word.append(0)
                    pass
            freq.append(freq_word)
    else:
        pass
    return freq
        
def COUNT_value():
    # we count the correspondences between directors and composers
    counter_film = []
    if len(Films) != 0:
        counter_film = []
        for k in range(len(Films)):
            counter_film.append(0)
            for i in Composers[k]:
                for j in composer:
                    if i == j:
                        counter_film[k] += 1
        
            for i in Directors[k]:
                for j in director:
                    if i == j:
                        counter_film[k] += 1
    else:
        pass
    return counter_film

def FINAL_SCORE():
    # we are going to sum all the frequence of the words into the docs and taking the mean
    if len(Films) != 0:
        Score_movie = [0 for i in range(len(Films))]
        for i in range(len(freq)):
            for j in range(len(Films)):
                Score_movie[j] += freq[i][j]
        # adding the special point if somthing match  
        for i in range(len(Score_movie)):
            Score_movie[i] = Score_movie[i]/len(Query)
            Score_movie[i] = Score_movie[i] + counter_film[i]*.25
    else:
        Score_movie = 0
    return Score_movie

def output_new_score(Films, input_path, Score_movie, Doc_Id_url, k, Film_score):
    Title = []
    Intro = []
    Url = []
    Score = []
    # for each film selected we are going to save title, intro and url
    for i in range(k,-1,-1):
        completeName_input = os.path.join(input_path, 'article_' + str(Film_score[Score_movie[i]]) + '.tsv')
        with open(completeName_input, 'r') as input_file:
            document = input_file.readline()
        document= document.split('\t')
        Title.append(document[0])
        Intro.append(document[1])
        Url.append(Doc_Id_url[Film_score[Score_movie[i]]])
        Score.append(Score_movie[i])
    #create a dictionary
    Data = {'title' : Title,
            'intro' : Intro,
            'url' : Url,
            'score' : Score
            }
   
    # make the dataframe 
    df = pd.DataFrame(data=Data)
    
    return df
    

def heapify(arr, n, i): 
    largest = i # Initialize largest as root 
    l = 2 * i + 1	 # left = 2*i + 1 
    r = 2 * i + 2	 # right = 2*i + 2 
    # See if left child of root exists and is 
    # greater than root 
    if l < n and arr[i] < arr[l]: 
        largest = l 
    # See if right child of root exists and is 
    # greater than root 
    if r < n and arr[largest] < arr[r]: 
        largest = r 
    # Change root, if needed 
    if largest != i: 
        arr[i],arr[largest] = arr[largest],arr[i] # swap 

        # Heapify the root. 
        heapify(arr, n, largest) 

# The main function to sort an array of given size 
def heapSort(arr): 
    n = len(arr) 

    # Build a maxheap. 
    for i in range(n, -1, -1): 
        heapify(arr, n, i) 

    # One by one extract elements 
    for i in range(n-1, 0, -1): 
        arr[i], arr[0] = arr[0], arr[i] # swap 
        heapify(arr, i, 0) 

# User research

In [ ]:
# input for the user
while True:
    try:
        Choice = int(input('which search: 1, 2 or 3?  '))
        if Choice == 3:
            
            Query = input('amazing: the query is?  ')
            # maybe somthing as 'do you want specify the director?'
            k = input("HOW MANY DOCUMENTS YOU WANT?\nhe defolt value is 3, press enter if you don't have a preference  ")
            if k == '':
                k = 2
            else:
                k = int(k)-1
            composer = input("CHOOSE THE MUSIC COMPOSER. \nPress enter if you don't have a preference  ")
            director = input("CHOOSE THE DIRECTOR? \nPress enter if you don't have a preference  ")
            
            
            pass
        else:
            Query = input('amazing: the query is?  ')
        break
    except:
        print('try again')

# filtering the query
Query = FILTER(Query) 

# initialize two arrays
All_docs = []
Docs = []




# if conjuntive query
if Choice == 1:
    # search all the rigth movies
    Films = conjuntive_query(Query, Inverted_index, Vocabulary, Docs)
    if len(Films) == 0:
        print('Sorry but there is nothing for you, try again')
    else:
        #make the dataframe
        df = output_conjuntive_query(Films, input_path, Doc_Id_url)
        print(df)

if Choice == 2:
    

if Choice == 3:
    # we star from the conjuntive query
    Films = conjuntive_query(Query, Inverted_index, Vocabulary, Docs)
    if len(Films) < k:
        k = len(Films) - 1
    
    composer = FILTER(composer)
    director = FILTER(director)
    
    
    #lets take the composer and the director
    Composers, Directors = search_composer_director(Films, input_path)
    
    # the score is based on the frequence of the words inside the documents
    freq = FREQ_value(Inverted_index, Vocabulary, Films)
    
    # if the user write sometring inside the director or the composer if the name compare inthe movie the score is 
    # If the user writes something in the section for the director and for the composer, for each word that matches the document the frequency count is increased by 25%
    #this is the counter of the matches
    counter_film = COUNT_value()
    
    # increasing the score
    Score_movie = FINAL_SCORE()
    if len(Films) != 0:
            
        Film_score = dict(zip(Score_movie, Films))
        
        # sort with the heap map
        heapSort(Score_movie) 
        
        # save the dataframe
        df = output_new_score(Films, input_path, Score_movie, Doc_Id_url, k, Film_score)
        print (df)
    pass


## Algorithm Question 4

#### Answer:
The Idea that is referred in this question is of the longest palindromic subsequence. I am defining this algorithm with the dynamic programming approach which is defined as dividing a problem into sub-problems, then solving these sub-problems in a way to store their results in a structure dataset for future query. First of all understand the problem, let’s suppose the sequence is “anwaralam” and its length is 9 and index 0-8. So,

First part is to identify the subsequences of this sequence, which can be defined as the sub-problem of the main - - problem.


Then second part is to identify their palindromic nature from first and last element of the subsequence and store the result in a data structure, (data structure is defined as a 2 dimensional table DST in our case)

Thirdly to identify the longest Palindromic Subsequence from the data structure of stored result

## The Detail ANSWER with Algorithm and Figures is available in Repository FILE: Answer_Algorithm_Question4.md

In [36]:
s = "ANWARALAM" # defined Sequence
nalgo4 = len(s)

In [34]:
## creating a table of rows and column equal to len of string
DST= [[0 for i in range(n)]for i in range (n)]
print (DST)

[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [37]:
print (nalgo4)

9


In [38]:
# for all values where length of sub sequence is 1
for i in range (nalgo4):
    DST[i][i] = 1
print (DST)

[[1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1]]


In [41]:
# for all values where length of subsequence is more than 1
# we make a loop for the lengths of sub-sequence    
for sseq in range(2, n+1): 
    for i in range(n-sseq+1):
        j = i+sseq-1
        if (s[i] == s[j] and sseq == 2): #conditioning for length 2 subsequences
            DST[i][j] = 2
        elif s[i] == s[j]:  #conditioning for all sseq greater than 2
            DST[i][j] = DST[i+1][j-1] + 2
        else:
            DST[i][j] = max(DST[i][j-1], DST[i+1][j])
    print (DST)
    


[[1, 1, 1, 3, 3, 3, 3, 5, 5], [0, 1, 1, 1, 1, 3, 3, 3, 3], [0, 0, 1, 1, 1, 3, 3, 3, 3], [0, 0, 0, 1, 1, 3, 3, 3, 3], [0, 0, 0, 0, 1, 1, 1, 3, 3], [0, 0, 0, 0, 0, 1, 1, 3, 3], [0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1]]
[[1, 1, 1, 3, 3, 3, 3, 5, 5], [0, 1, 1, 1, 1, 3, 3, 3, 3], [0, 0, 1, 1, 1, 3, 3, 3, 3], [0, 0, 0, 1, 1, 3, 3, 3, 3], [0, 0, 0, 0, 1, 1, 1, 3, 3], [0, 0, 0, 0, 0, 1, 1, 3, 3], [0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1]]
[[1, 1, 1, 3, 3, 3, 3, 5, 5], [0, 1, 1, 1, 1, 3, 3, 3, 3], [0, 0, 1, 1, 1, 3, 3, 3, 3], [0, 0, 0, 1, 1, 3, 3, 3, 3], [0, 0, 0, 0, 1, 1, 1, 3, 3], [0, 0, 0, 0, 0, 1, 1, 3, 3], [0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1]]
[[1, 1, 1, 3, 3, 3, 3, 5, 5], [0, 1, 1, 1, 1, 3, 3, 3, 3], [0, 0, 1, 1, 1, 3, 3, 3, 3], [0, 0, 0, 1, 1, 3, 3, 3, 3], [0, 0, 0, 0, 1, 1, 1, 3, 3], [0, 0, 0, 0, 0, 1, 1, 3, 3], [0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 

In [44]:
print(DST[0])

[1, 1, 1, 3, 3, 3, 3, 5, 5]


In [45]:
print("The Longest Palindromic sub Sequence is ", DST[0][n-1])

The Longest Palindromic sub Sequence is  5
